<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [184]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import warnings


warnings.filterwarnings('ignore')   #will remove warning messages

In [185]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

In [186]:
connection = psycopg2.connect(
    dbname = DBNAME,
    user = USER,
    password = ******,
    host = 'localhost,
    port = 5432
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [187]:
# текст запроса
query_3_1 = f'''select count (*)
            from public.vacancies
         '''

In [188]:
# результат запроса
print(pd.read_sql_query(query_3_1, connection))

   count
0  49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [189]:
# текст запроса
query_3_2 = f''' SELECT COUNT(DISTINCT id)
            FROM public.employers
         '''

In [190]:
# результат запроса
print(pd.read_sql_query(query_3_2, connection))

   count
0  23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [191]:
# текст запроса
query_3_3 = f'''SELECT COUNT (DISTINCT id)
            FROM public.areas
         '''

In [192]:
# результат запроса
print(pd.read_sql_query(query_3_3, connection))

   count
0   1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [193]:
# текст запроса
query_3_4 = f'''SELECT COUNT(DISTINCT id)
            FROM public.industries
         '''

In [194]:
# результат запроса
print(pd.read_sql_query(query_3_4, connection))

   count
0    294


***

In [195]:
# выводы по предварительному анализу данных

<span style="color:green">**О каких то выводах говорить слишком рано. У нас есть набор значений по количеству вакансий, общему количеству работодателей, количество регионов и количество сфер деятельности.
Всего размещено 49197 вакансий в 294 сферах деятельности. Зарегистрирован 23501 работодатель. Всего 1362 региона/населенных пункта.**</span>.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [196]:
# текст запроса
query_4_1 = f'''SELECT  pa.id, pa.name, COUNT (pv.id)
            FROM public.vacancies AS pv
            JOIN public.areas AS pa ON pv.area_id=pa.id
            GROUP BY pa.id
            ORDER BY COUNT (pv.id) DESC
            LIMIT 5
        '''

In [197]:
# результат запроса
print(pd.read_sql_query(query_4_1, connection))

     id             name  count
0     1           Москва   5333
1     2  Санкт-Петербург   2851
2  1002            Минск   2112
3     4      Новосибирск   2006
4   160           Алматы   1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [198]:
# текст запроса
query_4_2 = f'''SELECT COUNT(*)
            FROM public.vacancies
            WHERE salary_from IS NOT NULL or salary_to  IS NOT NULL
         '''


In [199]:
# результат
print(pd.read_sql_query(query_4_2, connection))

   count
0  24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [200]:
# текст запроса
query_4_3 = f'''SELECT ROUND(AVG(salary_from)), ROUND(AVG(salary_to))
            FROM public.vacancies
         '''


In [201]:
# результат запроса
print(pd.read_sql_query(query_4_3, connection))

     round     round
0  71065.0  110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [202]:
# текст запроса
query_4_4 = f'''SELECT schedule, employment, COUNT (id)
            FROM public.vacancies
            GROUP BY schedule, employment
            ORDER BY COUNT (id) DESC
            LIMIT 2
            '''

In [203]:
# результат запроса
print(pd.read_sql_query(query_4_4, connection))

           schedule        employment  count
0       Полный день  Полная занятость  35367
1  Удаленная работа  Полная занятость   7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [204]:
# текст запроса
query_4_5 = f'''SELECT experience, COUNT (id)
            FROM public.vacancies
            WHERE schedule= 'Удаленная работа' AND employment= 'Полная занятость'
            GROUP BY experience
            ORDER BY COUNT (id)
         '''


In [205]:
# результат запроса
print(pd.read_sql_query(query_4_5, connection))

           experience  count
0         Более 6 лет    218
1           Нет опыта    878
2       От 3 до 6 лет   2960
3  От 1 года до 3 лет   3746


***

In [206]:
# выводы по детальному анализу вакансий

<span style="color:green">**Ожидаемо что самое большое количество вакансий расположено в Москве и Санкт-Петербурге 5333 и 2851 соответственно. У 24073 вакансий заполнено хотя бы одно поле с зарплатой, что составляет 49% от общего количества вакансий. Анализ среднего значения зарплатной вилки практически бесполезен, так как большие зарплатные ожидания у столиц, и, естественно, что они будут тянуть эти показатели вверх. Всего 18% вакансий предполагают  удаленную работу. Основная масса вакансий подразумевает опыт работы от 1 до 6 лет**</span>.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [207]:
# текст запроса
query_5_1 = f'''SELECT  pe.id, pe.name, COUNT (pv.id)
            FROM public.vacancies AS pv
            JOIN public.employers AS pe ON pv.employer_id=pe.id
            GROUP BY pe.id
            ORDER BY COUNT (pv.id) DESC
            LIMIT 5
        '''


In [208]:
# результат запроса
print(pd.read_sql_query(query_5_1, connection))

      id           name  count
0   1740         Яндекс   1933
1   2748     Ростелеком    491
2  78638       Тинькофф    444
3   3529           СБЕР    428
4  39305  Газпром нефть    331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [209]:
# текст запроса
query_5_2 = f'''SELECT  pa.id, pa.name, COUNT (*)
            FROM public.areas AS pa
               LEFT JOIN public.employers AS pe ON pa.id=pe.area
                    LEFT JOIN public.vacancies AS pv ON pa.id=pv.area_id
                        WHERE pv.id IS NULL
            GROUP BY pa.id
            ORDER BY COUNT (*) desc
            LIMIT 1
        '''


In [210]:
# результат запроса
print(pd.read_sql_query(query_5_2, connection))

    id    name  count
0  113  Россия    410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [211]:
# текст запроса
query_5_3 = f'''SELECT  pe.id, pe.name, COUNT (DISTINCT pv.area_id)
            FROM public.vacancies AS pv
                JOIN public.employers AS pe ON pv.employer_id=pe.id
            GROUP BY pe.id
            ORDER BY COUNT (DISTINCT pv.area_id) desc
            LIMIT 1
        '''

In [212]:
# результат запроса
print(pd.read_sql_query(query_5_3, connection))

     id    name  count
0  1740  Яндекс    181


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [213]:
# текст запроса
query_5_4 = f'''SELECT   COUNT(pe.id)
            FROM public.employers AS pe
              LEFT JOIN public.employers_industries AS pei ON pe.id=pei.employer_id
                WHERE pei.employer_id IS NULL
        '''

In [214]:
# результат запроса
print(pd.read_sql_query(query_5_4, connection))

   count
0   8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [215]:
# текст запроса
query_5_5 = f'''SELECT   pe.id, pe.name, COUNT(*)
            FROM public.employers_industries AS pei
                JOIN public.employers AS pe ON pe.id=pei.employer_id
            GROUP BY pe.id
                HAVING COUNT(*)= 4
            ORDER BY pe.name
            LIMIT 3

        '''

In [216]:
# результат запроса
print(pd.read_sql_query(query_5_5, connection))

       id          name  count
0  800646  101 Интернет      4
1  763400      21vek.by      4
2   64174          2ГИС      4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [217]:
# текст запроса
query_5_6 = f'''SELECT  COUNT(*)
        FROM public.employers_industries AS pei
            JOIN public.industries AS pi on pei.industry_id=pi.id
                WHERE pi.name='Разработка программного обеспечения'
        '''

In [218]:
# результат запроса
print(pd.read_sql_query(query_5_6, connection))

   count
0   3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [219]:
# код для получения списка городов-милионников

url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(url,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'})
page = BeautifulSoup(response.text, 'html.parser')
soup=page.find('table', class_='standard sortable')
millions=''
for i in soup.find_all('a'):
    if i.text[0].upper()==i.text[0] and i.text[0]!='[':    #Exclude phrases that start with a small letter and '['
        millions+="'"+i.text+"'"+', '
millions=millions[:-2]
print(millions)

'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград'


In [220]:
# текст запроса
query_5_7 = f'''SELECT  pa.name AS city , COUNT(*) AS city_count
        FROM public.vacancies AS pv
            JOIN public.employers AS pe on pv.employer_id=pe.id
                JOIN public.areas AS pa on pv.area_id=pa.id
                    WHERE pe.name='Яндекс' AND pa.name IN ({millions})
        GROUP BY pa.name

        UNION ALL
        SELECT  'Total' AS city , COUNT(*) AS city_count
        FROM public.vacancies AS pv
            JOIN public.employers AS pe on pv.employer_id=pe.id
                JOIN public.areas AS pa on pv.area_id=pa.id
                    WHERE pe.name='Яндекс' AND pa.name IN ({millions})
        ORDER BY city_count
        '''

In [221]:
# результат запроса
print(pd.read_sql_query(query_5_7, connection))

               city  city_count
0              Омск          21
1         Челябинск          22
2        Красноярск          23
3         Волгоград          24
4             Пермь          25
5            Казань          25
6    Ростов-на-Дону          25
7               Уфа          26
8            Самара          26
9         Краснодар          30
10          Воронеж          32
11      Новосибирск          35
12  Нижний Новгород          36
13     Екатеринбург          39
14  Санкт-Петербург          42
15           Москва          54
16            Total         485


***

In [222]:
# выводы по анализу работодателей

<span style="color:green">**Самое большое количество вакансий размещено компанией Яндекс. Самое большое количество работодателей, по которым нет вакансий расположено в регионе, который в базе указан как  Россия. При анализе регионов присутствия видно, что Яндекс присутствует в 181 регионе.  8419 компаний не указали сферу деятельности. У 3553 работодателей указана сфера деятельности "Разработка ПО". Компанией Яндекс размещено 485 вакансий в городах-милллионниках из которых Москва и СПБ занимают лидирующие позиции - 54 и 42 соответственно.**</span>.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [223]:
#
query_6_1 = f'''SELECT  COUNT(*)
        FROM public.vacancies
            WHERE LOWER(name) LIKE '%data%' OR LOWER(name) LIKE '%данн%'
            '''


In [224]:
# результат запроса
print(pd.read_sql_query(query_6_1, connection))

   count
0   1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [225]:
# текст запроса
query_6_2 = f'''SELECT  COUNT(*)
        FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND  name NOT ILIKE '%html%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND
                    (name ILIKE '%junior%'
                    OR experience ILIKE '%нет опыта%'
                    OR employment ILIKE '%стажировка%')
            '''


In [226]:
# результат запроса
print(pd.read_sql_query(query_6_2, connection))

   count
0     51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [227]:
# текст запроса
query_6_3 = f'''SELECT  COUNT(*)
        FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND  name NOT ILIKE '%html%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                AND
                    (
                    key_skills ILIKE '%SQL%'
                    OR key_skills ILIKE '%postgres%'
                    )

            '''


In [228]:
# результат запроса
print(pd.read_sql_query(query_6_3, connection))

   count
0    201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [229]:
# текст запроса
query_6_4 = f'''SELECT  COUNT(*)
        FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND  name NOT ILIKE '%html%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
               AND
                   key_skills ILIKE '%python%'
            '''



In [230]:
# результат запроса
print(pd.read_sql_query(query_6_4, connection))

   count
0    351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [231]:
# текст запроса
## since the separator is ESC - the TAB character, then we remove it from the line
##and count the difference in characters plus one
query_6_5 = f''' SELECT AVG(CHAR_LENGTH(key_skills)+1 - CHAR_LENGTH(REPLACE(key_skills,CHR(9),'')))
         FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND  name NOT ILIKE '%html%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                    AND key_skills IS  NOT NULL
                '''

In [232]:
# результат запроса
print(round(pd.read_sql_query(query_6_5, connection),2))

    avg
0  6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [233]:
# текст запроса
query_6_6 = f'''SELECT  experience,  AVG((COALESCE(salary_from,salary_to,0)+COALESCE(salary_to,salary_from,0))/2)
        FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND  name NOT ILIKE '%html%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')
                    AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
        GROUP BY experience

            '''


In [234]:
# результат запроса
print(round(pd.read_sql_query(query_6_6, connection)))


           experience       avg
0           Нет опыта   74643.0
1  От 1 года до 3 лет  139675.0
2       От 3 до 6 лет  243115.0


***

In [235]:
# выводы по предметному анализу

<span style="color:green">**Из анализа средних зарплат заметно кратное увеличение зарплаты с ростом опыта работы. на позиции джуна средняя зарплата составляет порядка 75000 руб. Далее позиция миддла ценится уже в 2 раза выше. Опыт от 3 до 6 лет уже соответствует позиции сеньор или тимлидов. Для опыта выше 6 лет зарплата уже не указывается, так как устанавливается индивидульно. В среднем для Дата сайнтистов требуется 6-7 ключевых навыков. Навык владения  Пайтоном указываается в бОльшем количестве вакансий, нежели  SQL  и PosgreSQL**</span>.

# Общий вывод по проекту

In [236]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

# количество объявлений DS
query_6_7 = f'''SELECT  COUNT(*)
        FROM public.vacancies
            WHERE (name ILIKE '%data scientist%'
                OR name ILIKE '%data science%'
                OR name ILIKE '%исследователь данных%'
                OR (name LIKE '%ML%' AND  name NOT ILIKE '%html%')
                OR name ILIKE '%machine learning%'
                OR name ILIKE '%машинн%обучен%')

            '''


In [237]:
print(round(pd.read_sql_query(query_6_7, connection),2))
connection.close()

   count
0    480


<span style="color:green">**Общее количество вакансий по направленю DS  составляет 480 - это 27% всех предложений в сфере данных. Из них чуть более 10% предложений для  джунов. Средние зарплаты специалистов DS даже с небольшим опытом выше среднего значения верхней вилки всех объявлений
Можно продолжить провести анализ по городам для определения количества вакансий по DS в городах и размерах зарплат. Да много что еще можно. SQL интересен для получения выборок из баз, но все же обработку и анализ информации удобнее проводить в pd**</span>.